In [14]:
import re, string, unicodedata
import pandas as pd
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from helper import *
import warnings
warnings.filterwarnings('ignore')

In [15]:
stop = set(stopwords.words('english'))

In [16]:
data = pd.read_csv("psychiatry_psychology_mtsamples.csv")

In [17]:
data.head()

,sample_name,description,link,medical_reports
0,Adjustment Disorder & Encopresis,The patient was referred due to concerns regar...,https://www.mtsamples.com/site/pages/sample.as...,REASON FOR REFERRAL: The patient was referred ...
1,Agitation - ER Visit,Acute episode of agitation. She was complainin...,https://www.mtsamples.com/site/pages/sample.as...,CHIEF COMPLAINT: Agitation. HISTORY OF PRESENT...
2,Asperger Disorder,School reports continuing difficulties with re...,https://www.mtsamples.com/site/pages/sample.as...,SUBJECTIVE: School reports continuing difficul...
3,Attempted Suicide - Consult,"The patient is a 21-year-old Caucasian male, w...",https://www.mtsamples.com/site/pages/sample.as...,IDENTIFYING DATA: The patient is a 21-year-old...
4,Bipolar Affective Disorder - Consult,"The patient has a manic disorder, is presently...",https://www.mtsamples.com/site/pages/sample.as...,IDENTIFYING DATA: The patient is a 35-year-old...


In [18]:
data['text'] = data['sample_name'].str.cat(data['description'],sep=" ").str.cat(data['medical_reports'],sep=" ")
df = data
# df.head()

In [19]:
df = df.drop(df.columns[[0, 1, 2, 3]], axis=1)

In [20]:
df.head()

,text
0,Adjustment Disorder & Encopresis The patient w...
1,Agitation - ER Visit Acute episode of agitatio...
2,Asperger Disorder School reports continuing di...
3,Attempted Suicide - Consult The patient is a 2...
4,Bipolar Affective Disorder - Consult The patie...


In [21]:
df_list = df.values.tolist()

In [22]:
# df_list

In [23]:
import numpy as np
data_chunks = np.array_split(df_list, 10)

In [24]:
# preprocessing

def cleaning(str):
    #remove digit or numbers
#     str = re.sub(r"\b\d+\b", " ", str)
    #remove digit from string
#     str = re.sub("\S*\d\S*", "", str).strip()
#     remove digit
#     str = re.sub(r'[^a-z ]*([.0-9])*\d', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
#     remove numbers except inside words
#     str = re.sub(" \d+", " ", str)
    str = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", str)
    #to lowercase
    str = str.lower()
    #remove word with less than 2 alphabets
#     str = re.sub(r'\W*\b\w{1,2}\b', ' ', str)
    #Remove additional white spaces
#     str = re.sub('[\s]+', ' ', str)
    return str

#remove sentence which contains only 3 word\n",
def removeSentence(str):
    word = str.split()
    wordCount = len(word)
    if(wordCount<=3):
        str = ''
    return str

def removeStopword(str):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)
        
def preprocessing(str):
    str = cleaning(str)
    str = removeStopword(str)
    str = removeSentence(str)
    return str

In [25]:
cleaned = []

for text in df['text']:
    new_txt = preprocessing(text)
    cleaned.append(new_txt)

In [26]:
df_cleaned = pd.DataFrame(cleaned,columns=["text"])
df_cleaned.head()

,text
0,adjustment disorder encopresis patient referre...
1,agitation er visit acute episode agitation com...
2,asperger disorder school reports continuing di...
3,attempted suicide consult patient year old cau...
4,bipolar affective disorder consult patient man...


In [27]:
df_cleaned.to_csv("psy_cleaned.csv", index=False)